In [1]:
import pandas as pd
import re,itertools
from nltk.corpus import stopwords
flatten = itertools.chain.from_iterable
import spacy  
import numpy as np
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import Phrases

from gensim.models import Phrases

from gensim.models.phrases import Phraser

In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [3]:
stopword_list = stopwords.words('english')
stopword_list.extend(['google','facebook','twitter','linkedin','whatsapp'])

In [4]:
df = pd.read_csv('raw_blog_content_cleaned.csv')
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df

,title,url,body
0,Understanding Business Development,http://sethgodin.typepad.com/seths_blog/2009/0...,Understanding business development. Business D...
1,What are different types of Marketing Research?,http://whatismarketresearch.com/market-researc...,What are different types of Marketing Research...
2,5 Low-Cost Marketing Research Techniques for S...,https://www.credibly.com/incredibly/5-low-cost...,5 Low-Cost Marketing Research Techniques for S...
3,Are Product Managers the New Software Engineers?,https://hired.com/blog/highlights/product-mana...,Are Product Managers the New Software Engineer...
4,COMPETITIVE ANALYSIS PART 2: HOW TO ANALYZE C...,https://www.campaigncreators.com/blog/competit...,Competitive Analysis Part 2: How to Analyze Co...
...,...,...,...
110,The Basics of Customer Development for New Ent...,https://medium.com/get-greenlit/the-basics-of-...,The Basics of Customer Development for New Ent...
111,Do You Really Need Passion to Launch a Startup...,http://FI.co/posts/22201,When getting ready to embark on a major endeav...
112,What Employees Want From the Performance Manag...,https://www.td.org/insights/what-employees-wan...,"For employees, ongoing dialogue with their man..."
113,What Do Boards Actually Do?,https://bothsidesofthetable.com/what-do-boards...,What Do Boards Actually Do?\n\nThere’s a lot o...


In [5]:
def preProcess_stage1(text):
    #print "original:", text
    # sentence segmentation - assume already done
    # Remove Emails
    text = re.sub(r'\S*@\S*\s?', '', text)
    # Remove website links
    text = re.sub(r'http[s]?://\S+', '', text)
    # Remove distracting single quotes
    text = re.sub(r"\'", "", text)
    # Remove distracting double quotes
    text = re.sub(r'\"', "", text)
    # Remove new line characters
    text = re.sub(r'\s+', ' ', text)
    # word normalisation
    text = re.sub(r"(\w)([.,;:!?'/\"”\)])", r"\1 \2", text)
    text = re.sub(r"([.,;:!?'/\"“\(])(\w)", r"\1 \2", text)
    # normalisation
    text = re.sub(r"(\S)\1\1+",r"\1\1\1", text)
    #tokenising
    
    tokens = list(flatten([re.split(r"\s+",t) for t in re.split('(\d+)',text)]))
    tokens = [re.sub(r'[^A-Za-z]+','',t) for t in tokens]
    tokens = [t.lower() for t in tokens]
    tokens = [t for t in tokens if t not in ' ']
    return tokens

In [6]:
# corpus of all body data for NER
with open('corpus.txt','w') as f:
    allblog = df.body.values.tolist()
    for blog in allblog:
        sentList = []
        sentences = blog.split('.')
        for sent in sentences:
            tokens = preProcess_stage1(sent)
            sentList.append(' '.join(tokens))
        f.writelines('. '.join(sentList))

# stopword file for NER
stopword_file=open('stopword.txt','w')
stopword_file.writelines(stopword_list)
stopword_file.close()



In [7]:
df['tokens_1'] = df.apply(lambda x : preProcess_stage1(str(x.body)),axis = 1)

In [8]:
print(list(df['body'][0]))
#print(df['body'][0])

['U', 'n', 'd', 'e', 'r', 's', 't', 'a', 'n', 'd', 'i', 'n', 'g', ' ', 'b', 'u', 's', 'i', 'n', 'e', 's', 's', ' ', 'd', 'e', 'v', 'e', 'l', 'o', 'p', 'm', 'e', 'n', 't', '.', ' ', 'B', 'u', 's', 'i', 'n', 'e', 's', 's', ' ', 'D', 'e', 'v', 'e', 'l', 'o', 'p', 'm', 'e', 'n', 't', ' ', ' ', 'i', 's', ' ', 'a', ' ', 'm', 'y', 's', 't', 'e', 'r', 'i', 'o', 'u', 's', ' ', 't', 'i', 't', 'l', 'e', ' ', 'f', 'o', 'r', ' ', 'a', ' ', 'l', 'i', 't', 't', 'l', 'e', ' ', 'd', 'i', 's', 'c', 'u', 's', 's', 'e', 'd', ' ', 'f', 'u', 'n', 'c', 't', 'i', 'o', 'n', ' ', 'o', 'r', ' ', 'd', 'e', 'p', 'a', 'r', 't', 'm', 'e', 'n', 't', ' ', 'i', 'n', ' ', 'm', 'o', 's', 't', ' ', 'l', 'a', 'r', 'g', 'e', 'r', ' ', 'c', 'o', 'm', 'p', 'a', 'n', 'i', 'e', 's', '.', ' ', 'I', 't', '’', 's', ' ', 'a', 'l', 's', 'o', ' ', 'a', ' ', 'g', 'r', 'e', 'a', 't', ' ', 'w', 'a', 'y', ' ', 'f', 'o', 'r', ' ', 'a', 'n', ' ', 'e', 'n', 't', 'r', 'e', 'p', 'r', 'e', 'n', 'e', 'u', 'r', ' ', 'o', 'r', ' ', 's', 'm', 'a',

In [9]:
nlp = spacy.load('en', disable=['parser', 'ner'])

def remove_stopwords(tokens):
    return [t for t in tokens if t not in stopword_list]

def make_bigrams(tokens):
    tokens = [tokens]
    bigram = Phrases(tokens, min_count=1, threshold=2, delimiter=b' ')
    bigram_phraser = Phraser(bigram)
    bigram_tokens = list(flatten([bigram_phraser[sent] for sent in tokens]))
    bigrams_new = [t for t in bigram_tokens if len(t.split()) > 1 ]
#     bigrams_new = ['_'.join(t.split())  if len(t.split()) > 1 else t for t in bigram_tokens]
    return bigrams_new

def make_trigrams(tokens):
    tokens = [tokens]
    bigram = gensim.models.Phrases(tokens, min_count=1, threshold=2) 
    trigram = gensim.models.Phrases(bigram[tokens], threshold=2) 
    trigram_phraser = Phraser(trigram)
    trigram_tokens = list(flatten([trigram_phraser[sent] for sent in tokens]))
    trigrams_new = [t for t in trigram_tokens if len(t.split()) > 2 ]
    return trigrams_new

def lemmatization(tokens, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    doc = nlp(" ".join(tokens)) 
    return [token.lemma_ for token in doc if token.pos_ in allowed_postags]

def preProcess_stage2(tokens):
    # Stop Words
    clean_tokens = remove_stopwords(tokens)

    # BIgrams
    bigrams_tokens = make_bigrams(clean_tokens)

    # TRIgarms
    trigrams_tokens = make_trigrams(clean_tokens)
    
    # remove lemma with noun, adj, vb, adv words only
#     token_lemma = lemmatization(bigrams_tokens, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    token_lemma = lemmatization(clean_tokens, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    token_lemma = [t for t in token_lemma if len(t) > 2 ]
    return token_lemma + bigrams_tokens + trigrams_tokens

In [10]:
df['tokens_2'] = df.apply(lambda x : preProcess_stage2(x.tokens_1),axis = 1)

In [11]:
print(df['tokens_2'].head())

0    [understand, business, development, business, ...
1    [different, type, marketing, research, markete...
2    [marketing, research, technique, small, busine...
3    [product, manager, new, software, engineer, so...
4    [competitive, analysis, part, analyze, competi...
Name: tokens_2, dtype: object


In [12]:
data_lemmatized = df.tokens_2.values.tolist()
# data_lemmatized[:1]
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
# print(corpus[:1])

In [13]:
# id2word[0]

In [14]:
# # Human readable format of corpus (term-frequency)
# [[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

In [15]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [16]:
# Build LDA Multicore model
lda_multicore = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                           num_topics=20,
                           id2word=id2word,
                           workers=3)

In [17]:
# Build LDA Mallet model
mallet_path = 'mallet-2.0.8/bin/mallet' 
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

In [18]:
# # Print the Keyword in the 10 topics
# pprint(lda_model.print_topics())
# doc_lda = lda_model[corpus]

In [19]:
# Compute Perplexity of LDA
print('\nPerplexity of LDA: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score of LDA
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score of LDA: ', coherence_lda)

# Compute Perplexity of LDA Multicore
print('\nPerplexity of LDA Multicore: ', lda_multicore.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score of LDA Multicore
coherence_model_lda = CoherenceModel(model=lda_multicore, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score of LDA Multicore: ', coherence_lda)


# # Compute Perplexity of LDA Mallet
# print('\nPerplexity of LDA Mallet: ', ldamallet.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score of LDA Mallet
coherence_model_lda = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score of LDA Mallet: ', coherence_lda)



Perplexity of LDA:  -8.929030403953794


/home/nv/virt-py3tf/lib/python3.6/site-packages/scipy/sparse/lil.py:504: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/home/nv/virt-py3tf/lib/python3.6/site-packages/scipy/sparse/lil.py:506: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):



Coherence Score of LDA:  0.37448210465360676

Perplexity of LDA Multicore:  -9.344312891802419

Coherence Score of LDA Multicore:  0.3181399484222781

Coherence Score of LDA Mallet:  0.4350650178955216


In [20]:
# # Plotting tools
# import pyLDAvis
# import pyLDAvis.gensim  # don't skip this
# import matplotlib.pyplot as plt
# %matplotlib inline

In [21]:
# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
# vis

In [22]:
optimal_model = ldamallet
data = df.body.values.tolist()
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'body']

# Show
df_dominant_topic.head(50)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,body
0,0,10.0,0.2462,"business, development, skill, client, agency, ...",Understanding business development. Business D...
1,1,2.0,0.5189,"research, market, information, marketing, cust...",What are different types of Marketing Research...
2,2,2.0,0.3147,"research, market, information, marketing, cust...",5 Low-Cost Marketing Research Techniques for S...
3,3,11.0,0.1677,"manager, account, growth, year, management, em...",Are Product Managers the New Software Engineer...
4,4,1.0,0.5161,"market, industry, brand, competitor, competiti...",Competitive Analysis Part 2: How to Analyze Co...
5,5,0.0,0.5001,"conversion, website, test, testing, rate, traf...",7 Technical Skills All Growth Hackers Should L...
6,6,12.0,0.3084,"growth, lead, marketing, content, generation, ...",The Importance of Lead Generation | Criteria f...
7,7,2.0,0.6789,"research, market, information, marketing, cust...",Market Research Definition - Entrepreneur Smal...
8,8,8.0,0.7217,"work, team, project, process, week, agile, sof...",Kanban and Agile Kanban is a set of tools you...
9,9,5.0,0.3844,"business, make, money, create, network, work, ...",Product Development: Turning Months Into Weeks...


In [27]:
# print (pd.merge(df, df_dominant_topic, on='body'))
com = pd.merge(df, df_dominant_topic, on='body')

In [24]:
com.to_csv('lda_keyword.csv',index=False)

In [1]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

#vector = model['easy']
model.similarity('cat','dog')
model.most_similar('dog')

MemoryError: Unable to allocate array with shape (3000000, 300) and data type float32